In [1]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv
Saving scrape_test_edited_highlighted_final.csv to scrape_test_edited_highlighted_final.csv


In [2]:
import numpy as np
import pandas as pd
import io

training = pd.read_csv(io.BytesIO(uploaded['train.csv']))
testing = pd.read_csv(io.BytesIO(uploaded['scrape_test_edited_highlighted_final.csv']), encoding='latin1')

In [3]:
testing

,ï»¿keyword,text,target,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,ablaze,"Yesterday, at Igando, two men allegedly robbed...",1,NaN,NaN,NaN
1,ablaze,- NDLEA sets ablaze the N194 Billion worth of ...,1,NaN,NaN,NaN
2,ablaze,Putin's cancelled speech appears to have set M...,1,NaN,NaN,NaN
3,ablaze,"Seeing all over the country, people wearing ma...",0,NaN,NaN,NaN
4,ablaze,Death of a girl set ablaze in Dumka | Jharkhan...,1,NaN,NaN,NaN
...,...,...,...,...,...,...
2014,wrecked,This the second time I wrecked my car this yea...,0,NaN,NaN,NaN
2015,wrecked,@MikeBaggz Biden wrecked tens of thousands of ...,0,NaN,NaN,NaN
2016,wrecked,I do not like driving! My nerves be wrecked dr...,0,NaN,NaN,NaN
2017,wrecked,"iâm so sick and tired of work, this week rea...",0,NaN,NaN,NaN


In [4]:
import re
import string

# Credit to https://www.kaggle.com/tanulsingh077 (from https://www.kaggle.com/code/andreshg/nlp-glove-bert-tf-idf-lstm-explained#3.-Data-Pre-processing-%F0%9F%9B%A0)
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

training['clean_text'] = training['text'].apply(clean_text)
testing['clean_text'] = testing['text'].apply(clean_text)
testing.head()

,ï»¿keyword,text,target,Unnamed: 3,Unnamed: 4,Unnamed: 5,clean_text
0,ablaze,"Yesterday, at Igando, two men allegedly robbed...",1,NaN,NaN,NaN,yesterday at igando two men allegedly robbed a...
1,ablaze,- NDLEA sets ablaze the N194 Billion worth of ...,1,NaN,NaN,NaN,ndlea sets ablaze the billion worth of cocai...
2,ablaze,Putin's cancelled speech appears to have set M...,1,NaN,NaN,NaN,putins cancelled speech appears to have set mo...
3,ablaze,"Seeing all over the country, people wearing ma...",0,NaN,NaN,NaN,seeing all over the country people wearing mar...
4,ablaze,Death of a girl set ablaze in Dumka | Jharkhan...,1,NaN,NaN,NaN,death of a girl set ablaze in dumka jharkhand...


In [5]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

# Credit to https://www.kaggle.com/code/andreshg/nlp-glove-bert-tf-idf-lstm-explained#3.-Data-Pre-processing-%F0%9F%9B%A0

stop_words = stopwords.words('english')
extra_stopwords = ['u', 'im', 'c']
stop_words = stop_words + extra_stopwords

def remove_stopwords(text):
    tweet_as_list = text.split()
    removed_stopwords = []
    for word in tweet_as_list:
        if word not in stop_words:
            removed_stopwords.append(word)
    text = ' '.join(removed_stopwords)
    return text
    
training['clean_text'] = training['clean_text'].apply(remove_stopwords)
testing['clean_text'] = testing['clean_text'].apply(remove_stopwords)
testing.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,ï»¿keyword,text,target,Unnamed: 3,Unnamed: 4,Unnamed: 5,clean_text
0,ablaze,"Yesterday, at Igando, two men allegedly robbed...",1,NaN,NaN,NaN,yesterday igando two men allegedly robbed pos ...
1,ablaze,- NDLEA sets ablaze the N194 Billion worth of ...,1,NaN,NaN,NaN,ndlea sets ablaze billion worth cocaine recent...
2,ablaze,Putin's cancelled speech appears to have set M...,1,NaN,NaN,NaN,putins cancelled speech appears set moscow abl...
3,ablaze,"Seeing all over the country, people wearing ma...",0,NaN,NaN,NaN,seeing country people wearing maroon uvaldestr...
4,ablaze,Death of a girl set ablaze in Dumka | Jharkhan...,1,NaN,NaN,NaN,death girl set ablaze dumka jharkhand child we...


In [6]:
# Credit to https://www.kaggle.com/code/andreshg/nlp-glove-bert-tf-idf-lstm-explained#3.-Data-Pre-processing-%F0%9F%9B%A0

stemmer = nltk.SnowballStemmer("english")

def stemm_text(text):
    tweet_as_list = text.split()
    stemmed_as_list = []
    for word in tweet_as_list:
        stemmed_as_list.append(stemmer.stem(word))
    text = ' '.join(stemmed_as_list)
    return text

training['clean_text'] = training['clean_text'].apply(stemm_text)
testing['clean_text'] = testing['clean_text'].apply(stemm_text)
testing.head()

,ï»¿keyword,text,target,Unnamed: 3,Unnamed: 4,Unnamed: 5,clean_text
0,ablaze,"Yesterday, at Igando, two men allegedly robbed...",1,NaN,NaN,NaN,yesterday igando two men alleg rob pos attend ...
1,ablaze,- NDLEA sets ablaze the N194 Billion worth of ...,1,NaN,NaN,NaN,ndlea set ablaz billion worth cocain recent se...
2,ablaze,Putin's cancelled speech appears to have set M...,1,NaN,NaN,NaN,putin cancel speech appear set moscow ablaz an...
3,ablaze,"Seeing all over the country, people wearing ma...",0,NaN,NaN,NaN,see countri peopl wear maroon uvaldestrong itã...
4,ablaze,Death of a girl set ablaze in Dumka | Jharkhan...,1,NaN,NaN,NaN,death girl set ablaz dumka jharkhand child wel...


In [7]:
X_train = training['clean_text']
X_valid = testing['clean_text']
y_train = training['target']
y_valid = testing['target']

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vector = CountVectorizer(stop_words='english')
vector.fit(X_train)

X_train_trans = vector.transform(X_train)
X_valid_trans = vector.transform(X_valid)

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()

tfidf.fit(X_train_trans)
X_train_tfidf = tfidf.transform(X_train_trans)
X_valid_tfidf = tfidf.transform(X_valid_trans)

X_train_tfidf

<7613x13328 sparse matrix of type '<class 'numpy.float64'>'
	with 60693 stored elements in Compressed Sparse Row format>

In [10]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)

MultinomialNB()

In [11]:
from sklearn.metrics import roc_auc_score, confusion_matrix

auc_nb = roc_auc_score(y_valid, nb.predict_proba(X_valid_tfidf)[:, 1])
cm_nb = confusion_matrix(y_valid, nb.predict(X_valid_tfidf))

print("AUC:", auc_nb)
print("F1:", cm_nb[1][1] / (cm_nb[1][1] + 0.5 * (cm_nb[0][1] + cm_nb[1][0])))
print("Confusion Matrix:", cm_nb)

AUC: 0.8313753707088062
F1: 0.6542591267000716
Confusion Matrix: [[1079  255]
 [ 228  457]]
